In [3]:
import pandas as pd
import numpy as np
import datetime

from pytz import country_timezones
import pytz

import glob

In [4]:
from tqdm import tqdm
tqdm.pandas()

In [10]:
challenge_set = pd.read_csv("/mnt/SMB_share/mark/flight_competition/challenge_set.csv")
challenge_set["dataset"] = "challenge"
submission_set = pd.read_csv("/mnt/SMB_share/mark/flight_competition/final_submission_set.csv")
submission_set["dataset"] = "submission"
datasets = pd.concat([challenge_set, submission_set])

datasets = datasets[["flight_id", "aircraft_type"]]
tows = challenge_set[["flight_id", "tow"]]

## combine chunks by aircraft type

In [6]:
chunk_files = glob.glob("data/chunks/*")
traject_files = [file for file in chunk_files if "traject_all" in file]

In [10]:
for i, file in enumerate(traject_files): 
    print(i)
    df = pd.read_csv(file)
    df = df.merge(datasets, on="flight_id", how="left")
    these_types = df["aircraft_type"].unique()
    for this_type in these_types: 
        df[df["aircraft_type"]==this_type].to_parquet("data/by_type/"+this_type+"_"+str(i)+".parquet", index=False)

        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
by_type_files = glob.glob("data/by_type/*.parquet")

In [9]:
# replacers = {'B772': 'A333',
#  'B788': 'A332',
#  'A21N': 'A321',
#  'A20N': 'B738',
#  'B789': 'A333',
#  'CRJ9': 'E190',
#  'B77W': 'A333',
#  'BCS3': 'B737',
#  'AT76': 'E190',
#  'B763': 'A321',
#  'BCS1': 'B737',
#  'E195': 'E190',
#  'B38M': 'B738',
#  'A359': 'A333',
#  'B39M': 'B739',
#  'A343': 'A333',
#  'A310': 'A321',
#  'B752': 'A321',
#  'B773': 'A333',
#  'C56X': 'E190',
#  'E290': 'E190'}

In [11]:
# 

def split(a, n):
    k, m = divmod(len(a), n)
    #print(k,m)
    return [a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n)]

def split_aircraft(this_type, by_type_files):
    df = pd.DataFrame([])
    these_files = [x for x in by_type_files if this_type in x]
    for this_file in these_files: 
        this_df = pd.read_parquet(this_file)
        this_df = this_df[["sec_since_takeoff", "altitude", "groundspeed", "vertical_rate", "temperature", "specific_humidity", "tail_wind", "cross_wind", "flight_id", "aircraft_type"]]
        df = pd.concat([df, this_df])
    splits = split(list(df["flight_id"].unique()), 6)
    final_array = []
    [[final_array.append([i, x]) for x in array] for i, array in enumerate(splits)]
    split_df = pd.DataFrame(final_array, columns=["fold_group", "flight_id"])
    df["flight_id"] = df["flight_id"].astype(int)
    df = df.merge(split_df, on="flight_id", how="left")
    df = df.merge(tows, on="flight_id", how="left")
    return df
    
    

In [12]:
for this_type in datasets["aircraft_type"].unique():
    print(this_type)
    df = split_aircraft(this_type, by_type_files)
    df.to_csv("data/by_type_consolidated/"+this_type+".csv")

A320
A320
B772
B772
A333
A333
B788
B788
A21N
A21N
A20N
A20N
B738
B738
B789
B789
A321
A321
CRJ9
CRJ9
A319
A319
B77W
B77W
BCS3
BCS3
AT76
AT76
E190
E190
B763
B763
BCS1
BCS1
E195
E195
B38M
B38M
B739
B739
B737
B737
A359
A359
A332
A332
B39M
B39M
A343
A343
A310
A310
B752
B752
B773
B773
C56X
C56X
E290
E290
